In [1]:
import sys
sys.path.insert(1, '../')

In [2]:
import torch
from data.graph_dataset import OneDDatasetBuilder, OneDDatasetLoader
from data.pre_process import normalize, batchgraph_generation_wise
from data.post_process import print_1D
# from networks.gcnv1 import GCNV1
import matplotlib.pyplot as plt

In [3]:
# # Build dataset
# dataset = OneDDatasetBuilder(
#     raw_dir='/data1/tam/datasets',
#     root_dir='/data1/tam/downloaded_datasets_v2',
#     sub_dir='processed',
#     subjects='all',
#     refined_max_length=4.,
#     time_names=[str(i).zfill(3) for i in range(201)],
#     data_type = torch.float64,
# )

In [ ]:
# Load raw dataset
dataset = OneDDatasetLoader(
    
)